In this notebook we:
 1) do file conversions to generate .vtk files in order to generate more plots
 2) plot the loss from the training data

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import vtk
import os
import csv
import math
import matplotlib.pyplot as plt
from datetime import datetime

**CUDA**

In [2]:
len(tf.config.list_physical_devices('GPU'))>0

True

** .txt files generated from CompuCell3D to .csv**

In [ ]:
input_folder = "data/concentrations_txt/S8"
output_folder = "data/concentrations/S8"

# create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# iterate through files in input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        try:
            # construct input and output file paths
            input_filepath = os.path.join(input_folder, filename)
            output_filepath = os.path.join(output_folder, os.path.splitext(filename)[0] + ".csv")

            # read txt file and remove leading and trailing quotation marks from each line
            with open(input_filepath, 'r') as file:
                lines = [line.strip().strip('"') for line in file.readlines()]

            # convert to df and save as csv
            df = pd.DataFrame([line.split(',') for line in lines])
            df.to_csv(output_filepath, index=False, header=False, quoting=csv.QUOTE_NONE, escapechar=' ')
            print(f"Converted {input_filepath} to {output_filepath}")
        except Exception as e:
            print(f"Error converting {input_filepath}: {e}")


*csv to vtk for spatial plots*

In [59]:
def csv_to_vtk_per_timestep(csv_file, output_dir):
    try:
        df = pd.read_csv(csv_file)

        required_columns = ['timestep', 'X', 'Y']
        if not all(column in df.columns for column in required_columns):
            raise ValueError(f"CSV file must contain columns: {', '.join(required_columns)}")

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        timesteps = df['timestep'].unique()
        feature_names = [col for col in df.columns if col not in ['timestep', 'X', 'Y']]

        for timestep in timesteps:
            timestep_df = df[df['timestep'] == timestep]

            # Create VTK StructuredPoints object
            structured_points = vtk.vtkStructuredPoints()
            structured_points.SetDimensions(250, 250, 1)  # adjust
            structured_points.SetSpacing(1, 1, 1)
            structured_points.SetOrigin(0, 0, 0)

            # Add point data
            point_data = structured_points.GetPointData()
            for feature in feature_names:
                feature_array = vtk.vtkFloatArray()
                feature_array.SetName(feature)
                feature_array.SetNumberOfComponents(1)
                feature_array.SetNumberOfTuples(250 * 250)  # adjust

                for index, value in enumerate(timestep_df[feature]):
                    feature_array.SetValue(index, value)

                point_data.AddArray(feature_array)

            # Write the data to a VTK file
            vtk_file_path = os.path.join(output_dir, f"timestep_{timestep}.vtk")
            writer = vtk.vtkStructuredPointsWriter()
            writer.SetFileName(vtk_file_path)
            writer.SetInputData(structured_points)
            writer.Write()
            print(f"Successfully wrote {vtk_file_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

csv_file_path = "C:/Users/Giannis/Desktop/neural_agent_models_data/predictions/PINN(250x250)(72-100).csv"
output_dir = "C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)"
csv_to_vtk_per_timestep(csv_file_path, output_dir)

Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_72.vtk
Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_73.vtk
Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_74.vtk
Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_75.vtk
Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_76.vtk
Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_77.vtk
Successfully wrote C:/Users/Giannis/Desktop/neural_agent_models_data/vtk_outputs/vtk_outputs(250x250)/vtk_outputs(PINN 72-100)\timestep_78.vtk

*Merge data from different runs of different train/test splits*

In [53]:
def filter_timesteps(csv_file, output_file, max_timestep=10):
    try:
        # Read the CSV file
        df = pd.read_csv(csv_file)

        # Filter out rows with timestep >= max_timestep
        filtered_df = df[df['timestep'] < max_timestep]

        # Save the filtered DataFrame to a new CSV file
        filtered_df.to_csv(output_file, index=False)
        print(f"Successfully wrote filtered data to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

csv_file_path = "C:/Users/Giannis/Desktop/neural_agent_models_data/predictions/PINN(250x250)(72-89hrs).csv"
output_file_path = "C:/Users/Giannis/Documents/neural-agent-models-main/data/filtered_data.csv"
filter_timesteps(csv_file_path, output_file_path)

Successfully wrote filtered data to C:/Users/Giannis/Documents/neural-agent-models-main/data/filtered_data.csv


In [54]:
def merge_and_rename_timesteps(filtered_file, new_file, output_file):

    try:
    
        df_filtered = pd.read_csv(filtered_file)
        df_new = pd.read_csv(new_file)

        
        df_filtered['timestep'] = df_filtered['timestep'] + 72
        df_new['timestep'] = df_new['timestep'] + 82

        merged_df = pd.concat([df_filtered, df_new], ignore_index=True)

      
        merged_df.to_csv(output_file, index=False)
        print(f"Successfully wrote merged data to {output_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

filtered_file_path = "C:/Users/Giannis/Documents/neural-agent-models-main/data/filtered_data.csv"
new_file_path = "C:/Users/Giannis/Desktop/neural_agent_models_data/predictions/PINN(250x250)(82-100hrs).csv"
output_file_path = "C:/Users/Giannis/Desktop/neural_agent_models_data/predictions/PINN(250x250)(72-100).csv"
merge_and_rename_timesteps(filtered_file_path, new_file_path, output_file_path)

Successfully wrote merged data to C:/Users/Giannis/Desktop/neural_agent_models_data/predictions/PINN(250x250)(72-100).csv
